In [ ]:
#Lendo os dataframes
import pandas as pd
path = 'C:\\Users\\User\\Dropbox (CPI)\\EduardoMinsky\\Dados_Mineracao\\Dados Tabulares\\Dados_Relacionados\\'
#Pegando processos apenas do Pará
mun = pd.read_csv(path+'ProcessoMunicipio.csv')
substancia = pd.read_csv(path+'ProcessoSubstancia.csv')
eventos = pd.read_csv(path+'ProcessoEvento.csv')
processos = pd.read_csv(path+'Processo.csv')
pessoa = pd.read_csv(path+'ProcessoPessoa.csv')

## Preparando os DataFrames para 333 para as analises de área


In [ ]:
#Pegando para ouro
ouro = substancia.query('Substancia.str.contains("OURO")',engine = 'python')
#Pegando eventos 333,1402,513
eventos333 = eventos.query('IDEvento == 333')
#Filtrando os processos ativos:
prceossos_ativos = processos.query('BTAtivo=="S"')
#Merge entre processos ativos e processos de evento 333:
processos333_ativos = prceossos_ativos.merge(eventos333,how='inner',on='DSProcesso')
#Transformando o campo de data do evento333 em datetime
processos333_ativos['Data'] = pd.to_datetime(processos333_ativos.DTEvento).dt.year
#Filtrando os processos 333 ativos com a base de ouro:
processos333_ativos_ouro = processos333_ativos[processos333_ativos['DSProcesso'].isin(ouro.DSProcesso)]
#Retirando duplicados:
processos333_ativos_ouro.drop_duplicates(subset='DSProcesso' ,keep ='last',inplace=True)

#Pegando os estados para esses processo ativos de ouro
mun_ouro_ativo_333 = mun[mun['DSProcesso'].isin(processos333_ativos_ouro['DSProcesso'])]
#Agrupando os estados por Regiao (Norte, sul, Sudeste,CentroOeste, Nordeste)
sul =['RS','SC','PR']
sudeste = ['SP','MG','RJ','ES']
centrooeste =['MS','MT','GO','DF']
nordeste = ['BA','PI','MA','CE','RN','PB','PE','AL','SE']
norte = ['AC','RO','AM','RR','PA','TO','AP']
mun_ouro_ativo_333['Regiao'] = ['Sul' if x in sul else 'Sudeste' if x in sudeste else 'Centro_Oeste' if x in centrooeste else 'Nordeste' if x in nordeste else 'Norte' if x in norte else 'SemEstado' for x in mun_ouro_ativo_333['UF']]

#Dando Join com o df processos333_ativos_ouro para plotar o ano com regiao do brasil
join_333_regiao = processos333_ativos_ouro.merge(mun_ouro_ativo_333,on='DSProcesso',how='inner')
join_333_regiao.drop_duplicates(subset='DSProcesso' ,keep ='last',inplace=True)

#Transformando a coluna de Area para número!
import numpy as np
join_333_regiao['QTAreaHA'] = join_333_regiao['QTAreaHA'].str.replace(',', '.').astype(float)
join_333_regiao

## Plotando agora os mesmos gráficos mas para área 

In [ ]:
import seaborn as sns
import  matplotlib.pyplot as plt
%matplotlib inline
sns.set(rc={"figure.figsize":(17, 5)})

#Agrupando os valores de area por ano e somando
area_pedida_333 = join_333_regiao.groupby('Data').sum()

y = sns.barplot(
    data=area_pedida_333, x=area_pedida_333.index, y="QTAreaHA", capsize=.4, errcolor=".5",
    linewidth=3, edgecolor=".5", facecolor=(0, 0, 0, 0)
)
y.set(xlabel='Data', ylabel='Soma de Área')



plt.show()

In [ ]:
#DF Para figura 2
area_pedida_333.groupby('Data',as_index=True).QTAreaHA.sum().to_frame().reset_index().to_csv('Figura_2.csv')

In [ ]:
#DF Para figura 8
join_333_regiao.query('Data >= 2000').groupby('Data',as_index=False).QTAreaHA.sum().to_csv('Figura_8.csv')

# PLOTAR SEPARADO PARA REGIAO NORTE E CENTRO OESTE

In [ ]:
#Para Regiao Norte

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns


data_regiao_333= join_333_regiao.query('Data >= 2000').query('Regiao == "Norte" ').groupby(['Data','Regiao','UF'], as_index=False).sum()

palette_norte = {'PA':'black','RO':'b','AM':'g','AP':'y','RR':'orange','TO':'brown'}
matplotlib.rc_file_defaults()
ax1 = sns.set_style(style=None, rc=None )


fig, ax1 = plt.subplots(figsize=(15,5))
sns.pointplot(data=data_regiao_333.groupby(['Data'],as_index=False)['QTAreaHA'].sum(), x="Data", y="QTAreaHA",ax=ax1,
             color ='r')


ax2 = ax1.twinx()
sns.barplot(data=data_regiao_333, x="Data", y="QTAreaHA",hue='UF',dodge=True,palette = palette_norte,ax=ax1)
ax2.axhline(data_regiao_333.query('Data >= 2000')['QTAreaHA'].mean(),color ='cyan')

ax2.set(yticklabels=[])
plt.show()

In [ ]:
#Df para figura 9
a = data_regiao_333.groupby(['Data'],as_index=False)['QTAreaHA'].sum()

b = data_regiao_333.groupby(['Data','UF'],as_index=False)['QTAreaHA'].sum()
a.to_csv('Figura9_Linha.csv')
b.to_csv('Figura9_Barplot.csv')

In [ ]:
data_regiao_333.groupby(['UF'])['QTAreaHA'].sum().sort_values(ascending=True)
#data_regiao_333.QTAreaHA.sum()

In [ ]:
#Para Regiao Centro Oeste
data_regiao_333= join_333_regiao.query('Data >= 2000').query('Regiao == "Centro_Oeste" ').groupby(['Data','Regiao','UF'], as_index=False).sum()
data_regiao_333.groupby(['UF'])['QTAreaHA'].sum().sort_values(ascending=True)
#data_regiao_333.QTAreaHA.sum()

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns


data_regiao_333= join_333_regiao.query('Data >= 2000').query('Regiao == "Centro_Oeste" ').groupby(['Data','Regiao','UF'], as_index=False).sum()

palette_norte = {'MT':'black','GO':'b','MS':'g'}
matplotlib.rc_file_defaults()
ax1 = sns.set_style(style=None, rc=None )


fig, ax1 = plt.subplots(figsize=(15,5))
sns.pointplot(data=data_regiao_333.groupby(['Data'],as_index=False)['QTAreaHA'].sum(), x="Data", y="QTAreaHA",ax=ax1,
             color ='r')


ax2 = ax1.twinx()
sns.barplot(data=data_regiao_333, x="Data", y="QTAreaHA",hue='UF',dodge=True,palette = palette_norte,ax=ax1)


ax2.set(yticklabels=[])
plt.show()

In [ ]:
#Figura 10
c = data_regiao_333.groupby(['Data'],as_index=False)['QTAreaHA'].sum()

d = data_regiao_333.groupby(['Data','UF'],as_index=False)['QTAreaHA'].sum()
c.to_csv('Figura10_Linha.csv')
d.to_csv('Figura10_Barplot.csv')

## Plotando a área por ano mas quebrando para Regiao e um plot de Regiao quebrando por UF

## Preparando os dados para analisar area 513

In [ ]:
#Pegando para ouro
ouro = substancia.query('Substancia.str.contains("OURO")',engine = 'python')
#Pegando eventos 333,1402,513
eventos513 = eventos.query('IDEvento == 513')
#Filtrando os processos ativos:
prceossos_ativos = processos.query('BTAtivo=="S"')
#Merge entre processos ativos e processos de evento 333:
processos513_ativos = prceossos_ativos.merge(eventos513,how='inner',on='DSProcesso')
#Transformando o campo de data do evento513 em datetime
processos513_ativos['Data'] = pd.to_datetime(processos513_ativos.DTEvento).dt.year
#Filtrando os processos 513 ativos com a base de ouro:
processos513_ativos_ouro = processos513_ativos[processos513_ativos['DSProcesso'].isin(ouro.DSProcesso)]
#Retirando duplicados:
processos513_ativos_ouro.drop_duplicates(subset='DSProcesso' ,keep ='last',inplace=True)

#Pegando os estados para esses processo ativos de ouro
mun_ouro_ativo_513 = mun[mun['DSProcesso'].isin(processos513_ativos_ouro['DSProcesso'])]
#Agrupando os estados por Regiao (Norte, sul, Sudeste,CentroOeste, Nordeste)
sul =['RS','SC','PR']
sudeste = ['SP','MG','RJ','ES']
centrooeste =['MS','MT','GO','DF']
nordeste = ['BA','PI','MA','CE','RN','PB','PE','AL','SE']
norte = ['AC','RO','AM','RR','PA','TO','AP']
mun_ouro_ativo_513['Regiao'] = ['Sul' if x in sul else 'Sudeste' if x in sudeste else 'Centro_Oeste' if x in centrooeste else 'Nordeste' if x in nordeste else 'Norte' if x in norte else 'SemEstado' for x in mun_ouro_ativo_513['UF']]

#Dando Join com o df processos333_ativos_ouro para plotar o ano com regiao do brasil
join_513_regiao = processos513_ativos_ouro.merge(mun_ouro_ativo_513,on='DSProcesso',how='inner')
join_513_regiao.drop_duplicates(subset='DSProcesso' ,keep ='last',inplace=True)

#Transformando a coluna de Area para número!
import numpy as np
join_513_regiao['QTAreaHA'] = join_513_regiao['QTAreaHA'].str.replace(',', '.').astype(float)
full_join = join_513_regiao.merge(join_333_regiao,how='inner',on='DSProcesso')
full_join

In [ ]:
import seaborn as sns
import  matplotlib.pyplot as plt
%matplotlib inline
sns.set(rc={"figure.figsize":(17, 5)})

#Agrupando os valores de area por ano e somando
area_pedida_513 = full_join.query('Data_x >= 2000').groupby('Data_x').sum()

y = sns.barplot(
    data=area_pedida_513, x=area_pedida_513.index, y="QTAreaHA_x", capsize=.4, errcolor=".5",
    linewidth=3, edgecolor=".5", facecolor=(0, 0, 0, 0)
)
y.axhline(area_pedida_513['QTAreaHA_x'].mean(),color ='r')
y.set(xlabel='Data', ylabel='Soma de Área')
plt.show()

In [ ]:
full_join.query('Data_x >= 2000').QTAreaHA_x.sum()

In [ ]:
#Para Regiao Norte
data_regiao_513= full_join.query('Data_x >= 2000').query('Regiao_x == "Norte" ').groupby(['Data_x','Regiao_x','UF_x'], as_index=False).sum()
data_regiao_513.groupby(['UF_x'])['QTAreaHA_x'].sum().sort_values(ascending=True)
#data_regiao_513['QTAreaHA_x'].sum()
data_regiao_513.query('UF_x=="PA"').groupby(['Data_x']).QTAreaHA_x.sum().sort_values(ascending=True)

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns


data_regiao_513= full_join.query('Data_x >= 2000').query('Regiao_x == "Norte" ').groupby(['Data_x','Regiao_x','UF_x'], as_index=False).sum()

palette_norte = {'PA':'black','RO':'b','AM':'g','AP':'yellow','RR':'orange','TO':'brown'}
matplotlib.rc_file_defaults()
ax1 = sns.set_style(style=None, rc=None )


fig, ax1 = plt.subplots(figsize=(15,5))
sns.pointplot(data=data_regiao_513.groupby(['Data_x'],as_index=False)['QTAreaHA_x'].sum(), x="Data_x", y="QTAreaHA_x",ax=ax1,
             color ='r')


ax2 = ax1.twinx()
sns.barplot(data=data_regiao_513, x="Data_x", y="QTAreaHA_x",hue='UF_x',dodge=True,palette = palette_norte,ax=ax1)


ax2.set(yticklabels=[])
sns.move_legend(ax1, "upper right", title='UF')
plt.show()

In [ ]:
#Para Regiao Centro Oeste

data_regiao_513= full_join.query('Data_x >= 2000').query('Regiao_x == "Centro_Oeste" ').groupby(['Data_x','Regiao_x','UF_x'], as_index=False).sum()
data_regiao_513.groupby(['UF_x'])['QTAreaHA_x'].sum().sort_values(ascending=True)
#data_regiao_513['QTAreaHA_x'].sum()
data_regiao_513.query('UF_x=="MT"').groupby(['Data_x']).QTAreaHA_x.sum().sort_values(ascending=True)

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns


data_regiao_513= full_join.query('Data_x >= 2000').query('Regiao_x == "Centro_Oeste" ').groupby(['Data_x','Regiao_x','UF_x'], as_index=False).sum()

palette_norte = {'MT':'black','GO':'blue'}
matplotlib.rc_file_defaults()
ax1 = sns.set_style(style=None, rc=None )


fig, ax1 = plt.subplots(figsize=(15,5))
sns.pointplot(data=data_regiao_513.groupby(['Data_x'],as_index=False)['QTAreaHA_x'].sum(), x="Data_x", y="QTAreaHA_x",ax=ax1,
             color ='r')


ax2 = ax1.twinx()
sns.barplot(data=data_regiao_513, x="Data_x", y="QTAreaHA_x",hue='UF_x',dodge=True,palette = palette_norte,ax=ax1)


ax2.set(yticklabels=[])
sns.move_legend(ax1, "upper right", title='UF')
plt.show()

In [ ]:
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
matplotlib.rc_file_defaults()
ax1 = sns.set_style(style=None, rc=None )
sns.set(rc={"figure.figsize":(17, 5)})
#Analisando a Relação de quantidade de área permitida por Pessoa
pessoa_fisica = pessoa.query('Pessoa=="F"')
pessoa_juridica = pessoa.query('Pessoa=="J"')
p_fisica_only = pessoa_fisica[~pessoa_fisica['DSProcesso'].isin(pessoa_juridica['DSProcesso'])]
juridico_certo = pessoa[ ~ pessoa['DSProcesso'].isin(p_fisica_only['DSProcesso'])]
#Criando dummys
p_fisica_only['Pessoas'] = 'Pessoa Fisica'
juridico_certo['Pessoas'] = 'Pessoa Juridica'
p_fisica_only.drop_duplicates('DSProcesso',keep = 'first',inplace=True)
juridico_certo.drop_duplicates('DSProcesso',keep = 'first',inplace=True)
pessoas_certo = pd.concat([p_fisica_only,juridico_certo])

b = full_join.query('Data_x >=2000').merge(pessoas_certo,how='inner',on='DSProcesso')

b_pessoa = b.query('Data_x >= 2000').groupby(['Data_x','Pessoas','UF_x'], as_index=False)['QTAreaHA_x'].sum()
data_regiao_513= full_join.query('Data_x >= 2000').groupby(['Data_x','Regiao_x','UF_x'], as_index=False).sum()

fig, ax1 = plt.subplots(figsize=(15,5))
sns.pointplot(data=data_regiao_513.groupby(['Data_x'],as_index=False)['QTAreaHA_x'].sum(), x="Data_x", y="QTAreaHA_x",ax=ax1,
             color ='r')
sns.barplot(data=b_pessoa.groupby(['Data_x','Pessoas'],as_index=False).QTAreaHA_x.sum(), x="Data_x", y="QTAreaHA_x",hue='Pessoas',dodge=True,ax=ax1)

plt.show()

In [ ]:
#Separando as PJs em cooperativas e firmas individuais
contem_coop=[]
não_contem=[]
for i in range(len(b.query('Pessoas=="Pessoa Juridica"')['DSProcesso'].to_list())):
    num_proc = b.query('Pessoas=="Pessoa Juridica"')['DSProcesso'].to_list()[i]
    proc_filter = pessoa[pessoa['DSProcesso']==num_proc]
    nome_Requerente=proc_filter[proc_filter['TipoRelacao']=="Titular\Requerente"]['Pessoa_NOME'].to_list()[0].lower()
    if 'coo' in nome_Requerente:
        contem_coop.append(num_proc)
    else: não_contem.append(num_proc)
    
    
    
    


In [ ]:
#Tratando esses dados criados e unindo ao df principal
cooperativa = b[b['DSProcesso'].isin(contem_coop)]
cooperativa['Tipo'] = 'Pessoa Juridica - Cooperativa'


firma_individual = b[b['DSProcesso'].isin(não_contem)]
firma_individual['Tipo'] = 'Pessoa Juridica - Firma individual'

cooperativa_firma = pd.concat([cooperativa,firma_individual])
cooperativa_firma
fisica_true = b[~b['DSProcesso'].isin(cooperativa_firma['DSProcesso'])]
fisica_true['Tipo'] = 'Pessoa Fisica'

df_fisica_coop_individual = pd.concat([fisica_true,cooperativa_firma])


In [ ]:
#Plotando separando entre essas 3 classes de pessoas
df_por_tipo = df_fisica_coop_individual.query('Data_x >= 2000').groupby(['Data_x','Tipo','UF_x'], as_index=False)['QTAreaHA_x'].sum()

data_regiao_513= full_join.query('Data_x >= 2000').groupby(['Data_x','Regiao_x','UF_x'], as_index=False).sum()


fig, ax1 = plt.subplots(figsize=(15,5))
sns.pointplot(data=data_regiao_513.groupby(['Data_x'],as_index=False)['QTAreaHA_x'].sum(), x="Data_x", y="QTAreaHA_x",ax=ax1,
             color ='r')
sns.barplot(data=df_por_tipo.groupby(['Data_x','Tipo'],as_index=False).QTAreaHA_x.sum(), x="Data_x", y="QTAreaHA_x",hue='Tipo',dodge=True,ax=ax1)

plt.show()

In [ ]:
juridico_indiv = df_por_tipo.groupby(['Data_x','Tipo'],as_index=False).QTAreaHA_x.sum().query('Tipo=="Pessoa Juridica - Firma individual"').QTAreaHA_x.sum()
juridico_coop =  df_por_tipo.groupby(['Data_x','Tipo'],as_index=False).QTAreaHA_x.sum().query('Tipo=="Pessoa Juridica - Cooperativa"').QTAreaHA_x.sum()
fisica = df_por_tipo.groupby(['Data_x','Tipo'],as_index=False).QTAreaHA_x.sum().query('Tipo=="Pessoa Fisica"').QTAreaHA_x.sum()
print('Pessoa Juridica Individual:')
print(juridico_indiv)
print("################################")
print('Pessoa Juridica Cooperativa')
print(juridico_coop)
print("################################")
print('Pessoa Fisica')
print(fisica)

In [ ]:
df_fisica_coop_individual.query('Data_x >= 2000')

para = df_fisica_coop_individual.query('Data_x >= 2000').query('UF_x=="PA"')
mt = df_fisica_coop_individual.query('Data_x >= 2000').query('UF_x=="MT"')
np.quantile(mt['QTAreaHA_x'],0.90)#511.6260000000003
top10_mt = mt[mt['QTAreaHA_x'] > 511.6260000000003]
bottom_90mt = mt[mt['QTAreaHA_x'] < 511.6260000000003]
top10_mt
bottom_90mt


bottom_90mt.groupby('Tipo').Tipo.value_counts()

In [ ]:
np.quantile(para['QTAreaHA_x'],0.90)#100.15900000000002

top10_pa = para[para['QTAreaHA_x'] > 100.15900000000002]
bottom_90pa = para[para['QTAreaHA_x'] < 100.15900000000002]
bottom_90pa.Tipo.value_counts()

In [ ]:
df_por_tipo.groupby(['Data_x','Tipo'],as_index=False).QTAreaHA_x.sum().query('Data_x==2020 or Data_x==2021').groupby(['Data_x','Tipo'],as_index=False).QTAreaHA_x.sum()